# 作成した各種関数などをまとめるノート

In [ ]:
import copy
import glob
import japanize_matplotlib
import math
import matplotlib as mpl
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import sys
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import sklearn.preprocessing as sp
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [ ]:
# 各種値を宣言
benchmarks = ['bt', 'cg', 'ep', 'ft', 'is', 'lu', 'mg', 'sp']
classes = ["S", "W", "A", "B", "C", "D"]
processes = [1, 2, 4, 8, 16, 32, 64, 128, 256]
fix_process = 4
fix_benchmark_class = "C"

In [ ]:
def return_fixed_class(BenchMark="bt", Processes=[1, 2, 4, 8, 16, 32, 64, 128, 256], FixedBenchMarkClass="C"):
        path = './csv_files/'
        
        # fixed_Class にはベンチマーククラスFixedBenchMarkClassで実行プロセス数がProcessesに該当するものの結果が入る
        fixed_Class = []

        for process in Processes:
            file_name = ("pprof_"+BenchMark+FixedBenchMarkClass+str(process)+".csv")
            file_path = path+file_name
            if (os.path.exists(file_path) and os.stat(file_path).st_size != 0):
                data_frame = pd.read_csv(path+file_name)
                data_frame = data_frame.set_index(['Name'])
                fixed_Class.append(data_frame.rename(columns = {'#Call': process}).sort_index())
        return(fixed_Class)
    

In [ ]:
def show_fixed_class_graph(BenchMark="bt", Processes=[1, 2, 4, 8, 16, 32, 64, 128, 256], FixedBenchMarkClass="C"):

        markers = [".", ",", "o", "v", "^", "<", ">", "1", "2", "3", "4", "8", "s", "p", "*", "h", "H", "+", "x", "D", "d", "|", "_", "None", None, "", "$x$",
            "$\\alpha$", "$\\beta$", "$\\gamma$"]
        colors = ['#e41a1c', '#377eb8', '#4daf4a', '#984ea3', '#ff7f00', '#ffff33', '#a65628', '#f781bf'] 
        fixed_Class = return_fixed_class(BenchMark=BenchMark, Processes=Processes, FixedBenchMarkClass=FixedBenchMarkClass)
        if(len(fixed_Class) != 0):
            summary_fixed_Class = pd.concat(fixed_Class, axis=1)
            summary_fixed_Class.sort_index(axis=1, inplace=True)
            summary_fixed_Class_title = BenchMark+"においてベンチマーククラスをCに固定し実行プロセス数を変化させたときの実行された全ての関数のコール回数"

            x_axes = summary_fixed_Class.columns.tolist()

            y_axes = summary_fixed_Class.index.tolist()

            plt.figure()
            for y_axis in y_axes:
                color = random.choice(colors)
                label = y_axis
                marker = random.choice(markers)
                plt.plot(x_axes, summary_fixed_Class.T[y_axis], marker=marker, label=y_axis)
            plt.legend()
            plt.title(BenchMark+"_FixedBenchMarkClass="+FixedBenchMarkClass)
            plt.show()
            
# 使用例
# show_fixed_class_graph(BenchMark="cg")

In [ ]:
def return_fixed_process(BenchMark="bt", BenchMarkClasses=["S", "W", "A", "B", "C", "D"], FixedProcess=32):
    path = './csv_files/'

    # fixed_process には実行プロセス数が64でベンチマーククラスがA ~ Dまでの結果が入る
    fixed_process = list()

    for bench_mark_class in BenchMarkClasses:
        file_name = ("pprof_"+BenchMark+bench_mark_class+str(FixedProcess)+".csv")
        file_path = path+file_name
        if (os.path.exists(file_path) and os.stat(file_path).st_size != 0):
            data_frame = pd.read_csv(path+file_name)
            data_frame = data_frame.set_index(['Name'])
            fixed_process.append(data_frame.rename(columns = {'#Call': bench_mark_class}).sort_index())
    return(fixed_process)
       

In [ ]:
     
def show_fixed_process_graph(BenchMark="bt", BenchMarkClasses=["S", "W", "A", "B", "C", "D"], FixedProcess=32):
    fixed_process = return_fixed_process(BenchMark=BenchMark, BenchMarkClasses=BenchMarkClasses, FixedProcess=FixedProcess)
    markers = [".", ",", "o", "v", "^", "<", ">", "1", "2", "3", "4", "8", "s", "p", "*", "h", "H", "+", "x", "D", "d", "|", "_", "None", None, "", "$x$",
 "$\\alpha$", "$\\beta$", "$\\gamma$"]
    colors = ['#e41a1c', '#377eb8', '#4daf4a', '#984ea3', '#ff7f00', '#ffff33', '#a65628', '#f781bf']
    if (len(fixed_process) != 0):
            summary_fixed_process = pd.concat(fixed_process, axis=1)

            x_axes = summary_fixed_process.columns.tolist()

            y_axes = summary_fixed_process.index.tolist()

            plt.figure()
            for y_axis in y_axes:
                color = random.choice(colors)
                label = y_axis
                marker = random.choice(markers)
                plt.plot(x_axes, summary_fixed_process.T[y_axis], marker=marker, label=y_axis)
            plt.legend()
            plt.title(BenchMark+"_FixedProcess="+str(FixedProcess))
            plt.show()
            
# 使用例            
# show_fixed_process_graph(BenchMark="cg")

In [ ]:
def show_graph(BenchMarks=[], Processes=[], BenchMarkClasses=[], fix_process=4, fix_benchmark_class="C"):

    if (BenchMarks==[] or Processes==[] or BenchMarkClasses==[]):
        print("関数の引数となっている配列が空です。")
    else:
        for bench_mark in BenchMarks:
            show_fixed_class_graph(BenchMark=bench_mark, Processes=Processes, FixedBenchMarkClass=fix_benchmark_class)
            show_fixed_process_graph(BenchMark=bench_mark, BenchMarkClasses=BenchMarkClasses, FixedProcess=fix_process)            

bench_marks = ['bt', 'cg', 'ep', 'ft', 'is', 'lu', 'mg', 'sp']
processes = [1, 2, 4, 8, 16, 32, 64, 128, 256]
classes = ["S", "W", "A", "B", "C", "D"]

# 使用例
# show_graph(bench_marks, processes, classes)

In [ ]:
def return_summarized_Fixed_dataframe(BenchMark_name = "cg", fixed="class"):

    def evaluate_dataframes(df1, df2):
        for i in range(len(df1.values.tolist()[0])):
            if(df1.values.tolist()[0][i] != df2.values.tolist()[0][i]):
                return False
        return True
    fixed_df = 0
    if (fixed == "class"):
        fixed_df = return_fixed_class(BenchMark=BenchMark_name)
    elif (fixed == "process"):
        fixed_df = return_fixed_process(BenchMark=BenchMark_name)
    summary_fixed_df = pd.concat(fixed_df, axis=1)
    dropped_summary_fixed_df = summary_fixed_df.drop_duplicates()
    dropped_summary_fixed_df_renamed = dropped_summary_fixed_df

    for dropped_index_name in dropped_summary_fixed_df.index.values:
        dropped_index_name_data = summary_fixed_df.loc[[dropped_index_name]]
        replace_name = dropped_index_name
        for all_index_name in summary_fixed_df.index.values:
            all_index_name_data = summary_fixed_df.loc[[all_index_name]]
            if(dropped_index_name == all_index_name):
                pass
            elif(evaluate_dataframes(dropped_index_name_data, all_index_name_data)):
                replace_name += f", {all_index_name}"
        dropped_summary_fixed_df_renamed = dropped_summary_fixed_df_renamed.rename(index={dropped_index_name: replace_name})
    
    return dropped_summary_fixed_df_renamed
    

In [ ]:
# 平均絶対パーセント誤差 (MAPE)(Mean Absolute Percent Error (MAPE))を返す関数
# 引数として長さの同じ二つのリストをとる
# 引数l1: 実測値のリスト
# 引数l2: 予測値のリスト

def mape_score(l1, l2):
    return_num = 0
    if(len(l1) != len(l2)):
        print("引数のリストの長さが異なります", end=", ")
        return -1
    for i in range(len(l1)):
        l1_num = l1[i]
        l2_num = l2[i]
        
        return_num += abs((l1_num - l2_num)/l1_num)

    return_num /= len(l1)
    return_num *= 100
    return return_num

# 使用例：mape_score([1,2,3,4], [4,3,2,1])
type(mape_score([1,2,3,4], [4,3,2,1]))

In [ ]:
def split_by_ratio(base_list :list, test_ratio :float):
    test_index = math.floor(len(base_list) * float(test_ratio))
    train_list = base_list[:-test_index]
    test_list = base_list[-test_index:]
    return train_list, test_list

class ModelBase:
    def __init__(self, raw_x, raw_y, benchmark_name="benchmark_name", function_name="function_name", test_ratio=0.3):
        self.benchmark_name = benchmark_name
        self.function_name = function_name
        self.xlabel = "実行時のプロセス数"
        self.ylabel = "関数のコール回数"

        self.raw_x = np.reshape(raw_x, (-1, 1))
        self.raw_y = np.reshape(raw_y, (-1, 1))
        self.train_x, self.test_x = split_by_ratio(self.raw_x, test_ratio)
        self.train_y, self.test_y = split_by_ratio(self.raw_y, test_ratio)
        
        if(len(self.train_x) == 0 or len(self.train_y) == 0 or len(self.test_x) == 0 or len(self.test_y) == 0):
            print(f"学習用とテスト用にデータを分割するのに問題が生じています。@{benchmark_name}")
            print(f"len(self.train_x) == {len(self.train_x)}")
            print(f"len(self.train_y) ==  {len(self.train_y)}")
            print(f"len(self.test_x) == {len(self.test_x)}")
            print(f"len(self.test_y) == {len(self.test_y)}")
        
        self.x_model_line = np.random.rand(1024, 1) * self.raw_x.max()
        self.x_model_line.sort(axis=0)
        self.y_model_line = 0
        
        self.lr = 0
        self.r2_score = 0
        
    def calc_lr(self):
        self.lr = 0
     
    def calc_r2_score(self):
        self.r2_score = 0
        
    def calc_mae_score(self):
        self.mae_score = 0
        
    def calc_mse_score(self):
        self.mse_score = 0
        
    def calc_rmse_score(self):
        self.rmse_score = 0
    
    def calc_mape_score(self):
        self.mape_score = 0
        
    def plot_graph(self):
        plt.figure()
        plt.plot(self.raw_x, self.raw_y, color="red")
        plt.show()
        

In [ ]:
class ModelLin(ModelBase):
    def calc_lr(self, AllData=False):
        self.lr = LinearRegression()
        self.lr.fit(self.train_x, self.train_y)
        
    def calc_r2_score(self):
        test_y_predicted = self.lr.predict(self.test_x)
        self.r2_score = r2_score(self.test_y, test_y_predicted)
        
    def calc_mae_score(self):
        test_y_predicted = self.lr.predict(self.test_x)
        self.mae_score = mean_absolute_error(self.test_y, test_y_predicted)
        
    def calc_mse_score(self):
        test_y_predicted = self.lr.predict(self.test_x)
        self.mse_score = mean_squared_error(self.test_y, test_y_predicted)
        
    def calc_rmse_score(self):
        self.calc_mse_score()
        self.rmse_score = np.sqrt(self.mse_score)
        
    def calc_mape_score(self):
        test_y_predicted = self.lr.predict(self.test_x)
        self.mape_score = float(mape_score(self.test_y, test_y_predicted))

    def calc_mape_score_InTrain(self):
        train_y_predicted = self.lr.predict(self.train_x)
        self.mape_score_InTrain = float(mape_score(self.train_y, train_y_predicted))
        
    def plot_graph(self):
        plt.figure()
        plt.scatter(self.raw_x, self.raw_y)
        self.y_model_line = self.lr.predict(self.x_model_line)
        plt.plot(self.x_model_line, self.y_model_line, color="red")
        plt.xlabel(self.xlabel)
        plt.ylabel(self.ylabel)
        
    def predict(self, num):
        predicted = self.lr.predict(num)
        return(predicted)

In [ ]:
def inverter_log10_func(x):
    return 10**x

class ModelLog10(ModelBase):
    
    def calc_lr(self, AllData=False):
        self.transformer_log10 = sp.FunctionTransformer(func=np.log10, inverse_func=inverter_log10_func)
        x_train_log10 = self.transformer_log10.transform(self.train_x)
        y_train_log10 = self.transformer_log10.transform(self.train_y)
        
        self.lr = LinearRegression()
        self.lr.fit(x_train_log10, y_train_log10)
        
    def calc_r2_score(self):
        train_x_log10 = self.transformer_log10.transform(self.train_x)
        train_y_predicted_log10 = self.lr.predict(train_x_log10)
        train_y_predicted = self.transformer_log10.inverse_transform(train_y_predicted_log10)
        self.r2_score = r2_score(self.train_y, train_y_predicted)
        
    def calc_mae_score(self):
        train_x_log10 = self.transformer_log10.transform(self.train_x)
        train_y_predicted_log10 = self.lr.predict(train_x_log10)
        train_y_predicted = self.transformer_log10.inverse_transform(train_y_predicted_log10)
        self.mae_score = mean_absolute_error(self.train_y, train_y_predicted)
        
    def calc_mse_score(self):
        train_x_log10 = self.transformer_log10.transform(self.train_x)
        train_y_predicted_log10 = self.lr.predict(train_x_log10)
        train_y_predicted = self.transformer_log10.inverse_transform(train_y_predicted_log10)
        self.mse_score = mean_squared_error(self.train_y, train_y_predicted)
        
    def calc_rmse_score(self):
        self.calc_mse_score()
        self.rmse_score = np.sqrt(self.mse_score)
        
    def calc_mape_score(self):
        test_x_log10 = self.transformer_log10.transform(self.test_x)
        test_y_predicted_log10 = self.lr.predict(test_x_log10)
        test_y_predicted = self.transformer_log10.inverse_transform(test_y_predicted_log10)
        self.mape_score = float(mape_score(self.test_y, test_y_predicted))

    def calc_mape_score_InTrain(self):
        train_x_log10 = self.transformer_log10.transform(self.train_x)
        train_y_predicted_log10 = self.lr.predict(train_x_log10)
        train_y_predicted = self.transformer_log10.inverse_transform(train_y_predicted_log10)
        self.mape_score_InTrain = float(mape_score(self.train_y, train_y_predicted))
        
    def plot_graph(self):
        plt.figure()
        plt.scatter(self.raw_x, self.raw_y)
        x_model_line_log10 = self.transformer_log10.transform(self.x_model_line)
        y_model_line_log10 = self.lr.predict(x_model_line_log10)
        self.y_model_line = self.transformer_log10.inverse_transform(y_model_line_log10)
        plt.plot(self.x_model_line, self.y_model_line, color="red")
        plt.xlabel(self.xlabel)
        plt.ylabel(self.ylabel)
    
    def predict(self, num):
        num_log10 = self.transformer_log10.transform(num)
        predicted_log10 = self.lr.predict(num_log10)
        predicted = self.transformer_log10.inverse_transform(predicted_log10)
        return(predicted)

In [ ]:
# Inverse proportion
def ip_func(x):
    return 1/x

class ModelIP(ModelBase):
    
    def calc_lr(self, AllData=False):
        self.transformer_ip = sp.FunctionTransformer(func=ip_func, inverse_func=ip_func)
        y_train_ip = self.transformer_ip.transform(self.train_y)        
        self.lr = LinearRegression()
        self.lr.fit(self.train_x, y_train_ip)
        
    def calc_r2_score(self):
        train_y_predicted_ip = self.lr.predict(self.train_x)
        train_y_predicted = self.transformer_ip.inverse_transform(train_y_predicted_ip)
        self.r2_score = r2_score(self.train_y, train_y_predicted)
        
    def calc_mae_score(self):
        train_y_predicted_ip = self.lr.predict(self.train_x)
        train_y_predicted = self.transformer_ip.inverse_transform(train_y_predicted_ip)
        self.mae_score = mean_absolute_error(self.train_y, train_y_predicted)
        
    def calc_mse_score(self):
        train_y_predicted_ip = self.lr.predict(self.train_x)
        train_y_predicted = self.transformer_ip.inverse_transform(train_y_predicted_ip)
        self.mse_score = mean_squared_error(self.train_y, train_y_predicted)
        
    def calc_rmse_score(self):
        self.calc_mse_score()
        self.rmse_score = np.sqrt(self.mse_score)
        
    def calc_mape_score(self):
        test_y_predicted_ip = self.lr.predict(self.test_x)
        test_y_predicted = self.transformer_ip.inverse_transform(test_y_predicted_ip)
        self.mape_score = float(mape_score(self.test_y, test_y_predicted))

    def calc_mape_score_InTrain(self):
        train_y_predicted_ip = self.lr.predict(self.train_x)
        train_y_predicted = self.transformer_ip.inverse_transform(train_y_predicted_ip)
        self.mape_score_InTrain = float(mape_score(self.train_y, train_y_predicted))
        
    def plot_graph(self):
        plt.figure()
        plt.scatter(self.raw_x, self.raw_y)
        y_model_line_ip = self.lr.predict(self.x_model_line)
        self.y_model_line = self.transformer_ip.inverse_transform(y_model_line_ip)
        plt.plot(self.x_model_line, self.y_model_line, color="red")
        plt.xlabel(self.xlabel)
        plt.ylabel(self.ylabel)
        
    def predict(self, num):
        predicted_ip = self.lr.predict(num)
        predicted = self.transformer_ip.inverse_transform(predicted_ip)
        return(predicted)

In [ ]:
class ModelBranch(ModelBase):
    
    def calc_lr(self, AllData=False):
        # 後述する t を算出するための処理
        max_in_train_y = max(self.train_y)
        max_in_train_y_first_index = self.train_y.tolist().index(max_in_train_y)
        # 分岐点のインデックスを t とする
        t = max_in_train_y_first_index
        self.t = t
        if(self.t == 0 or self.t == len(self.train_y) - 1):
            self.lr1 = LinearRegression()
            self.lr1.fit(self.train_x, self.train_y)
            self.lr2 = LinearRegression()
            self.lr2.fit(self.train_x, self.train_y)
        else:
            self.x_train_1 = self.train_x[:t]
            self.x_train_2 = self.train_x[t:]
            self.y_train_1 = self.train_y[:t]
            self.y_train_2 = self.train_y[t:]
            self.lr1 = LinearRegression()
            self.lr1.fit(self.x_train_1, self.y_train_1)
            self.lr2 = LinearRegression()
            self.lr2.fit(self.x_train_2, self.y_train_2)
        
    def calc_mape_score(self):
        if(self.t == 0 or self.t == len(self.train_y) - 1):
            test_y_predicted = self.lr1.predict(self.test_x)
            self.mape_score = float(mape_score(self.test_y, test_y_predicted))
        else:
            x_test = self.test_x
            y_test = self.test_y
            y_test_predicted = self.lr2.predict(x_test)
            self.mape_score = float(mape_score(y_test, y_test_predicted))

    def calc_mape_score_InTrain(self):
        if(self.t == 0 or self.t == len(self.train_y) - 1):
            train_y_predicted = self.lr1.predict(self.train_x)
            self.mape_score_InTrain = float(mape_score(self.train_y, train_y_predicted))
        else:
            train_y_predicted_1 = self.lr1.predict(self.x_train_1)
            train_y_predicted_2 = self.lr2.predict(self.x_train_2)
            mape_1 = float(mape_score(self.y_train_1, train_y_predicted_1))
            mape_2 = float(mape_score(self.y_train_2, train_y_predicted_2))
            self.mape_score_InTrain = (mape_1 + mape_2) / 2
        
    def plot_graph(self):
        plt.figure()
        plt.scatter(self.raw_x, self.raw_y)

        if(self.t == 0 or self.t == len(self.train_y) - 1):
            y_model_line = self.lr.predict(x_model_line)
            plt.plot(self.x_model_line, y_model_line, color="red")
        else:
            # 回帰曲線を二つのモデルで分割するための処理
            x_model_line = self.x_model_line
            t_in_model_line = 0
            for i in range(len(x_model_line)):
                if (self.train_x[self.t] < x_model_line[i]):
                    t_in_model_line = i
                    break
                else:
                    t_in_model_line = i
            
            x_model_line1 = self.x_model_line[:t_in_model_line]
            x_model_line2 = self.x_model_line[t_in_model_line:]
            y_model_line1 = self.lr1.predict(x_model_line1)
            y_model_line2 = self.lr2.predict(x_model_line2)

            plt.plot(x_model_line1, y_model_line1, color="red")
            plt.plot(x_model_line2, y_model_line2, color="red")
    #         plt.plot(self.test_x, self.test_y, color="yellow")
        plt.xlabel(self.xlabel)
        plt.ylabel(self.ylabel)
        
    def predict(self, num):
        predicted = self.lr2.predict(num)
        return(predicted)

In [ ]:

def return_dict_summary_fixed(benchmark_name="cg", fixed="class"):
    if (fixed == "class"):
        fixed_ = return_fixed_class(BenchMark=benchmark_name)
    else:
        fixed_ = return_fixed_process(BenchMark=benchmark_name)
    summary_fixed_ = pd.concat(fixed_ , axis=1)
    columns = summary_fixed_.columns.to_numpy()
    index = summary_fixed_.index.to_numpy()
    if(fixed == "class"):
        dict_summary_fixed_ = {"processes":columns}
    else:
        dict_summary_fixed_ = {"class":columns}
    for index_name in index:
        dict_summary_fixed_[index_name] = summary_fixed_.T[index_name].to_numpy()
    return dict_summary_fixed_

# NaNが入った引数のリストをNaNのみを0にして返す関数
def return_non_NaN_list(target_list):
    for i in range(len(target_list)):
        if (math.isnan(target_list[i])):
            target_list[i] = 0
    return target_list
# NaNが入ったリストが引数として渡されるとTrueを返す関数
def does_include_nan(target_list):
    for i in range(len(target_list)):
        if(math.isnan(target_list[i])):
            return True
    return False

In [ ]:
# 線形・対数・反比例モデルでフィッティングを行い、MAPE値をまとめたCSVファイルを作成する関数
def generateScoreTable(benchmark_name="cg"):
    list_ScoreTable = []
    dict_summary_fixed_class = return_dict_summary_fixed(benchmark_name=benchmark_name, fixed="class")
    raw_x = dict_summary_fixed_class["processes"]
    for content in dict_summary_fixed_class:
        if(content == "processes"):
            continue
        raw_y = dict_summary_fixed_class[content]
        if(does_include_nan(raw_y)):
            continue
        # 線形モデル
        model_lin = ModelLin(raw_x, raw_y, benchmark_name, content)
        model_lin.calc_lr()
        model_lin.calc_r2_score()
        model_lin.calc_mae_score()
        model_lin.calc_mse_score()
        model_lin.calc_rmse_score()
        model_lin.calc_mape_score()
        # logモデル
        model_log10 = ModelLog10(raw_x, raw_y, benchmark_name, content)
        model_log10.calc_lr()
        model_log10.calc_r2_score()
        model_log10.calc_mae_score()
        model_log10.calc_mse_score()
        model_log10.calc_rmse_score()
        model_log10.calc_mape_score()
        # 反比例モデル
        model_ip = ModelIP(raw_x, raw_y, benchmark_name, content)
        model_ip.calc_lr()
        model_ip.calc_r2_score()
        model_ip.calc_mae_score()
        model_ip.calc_mse_score()
        model_ip.calc_rmse_score()
        model_ip.calc_mape_score()
        
        list_ScoreTable.append([content, model_lin.mape_score, model_log10.mape_score, model_ip.mape_score])
    df_ScoreTable = pd.DataFrame(list_ScoreTable)
    df_ScoreTable.columns = ["", "x mape", "logx mape", "1/x mape"]
    df_ScoreTable.set_index("",inplace=True)
    df_ScoreTable.to_csv("./tmp_GenerateScoreTable/"+benchmark_name+".csv")

In [ ]:
# 行の内容が同じものをまとめ、行タイトルに重複した行タイトルがまとめられた、データフレームが返される関数

def return_summarized_Fixed_dataframe(BenchMark_name = "cg", fixed="class"):

    def evaluate_dataframes(df1, df2):
        for i in range(len(df1.values.tolist()[0])):
            if(df1.values.tolist()[0][i] != df2.values.tolist()[0][i]):
                return False
        return True
    fixed_df = 0
    if (fixed == "class"):
        fixed_df = return_fixed_class(BenchMark=BenchMark_name)
    elif (fixed == "process"):
        fixed_df = return_fixed_process(BenchMark=BenchMark_name)
    summary_fixed_df = pd.concat(fixed_df, axis=1)
    dropped_summary_fixed_df = summary_fixed_df.drop_duplicates()
    dropped_summary_fixed_df_renamed = dropped_summary_fixed_df

    for dropped_index_name in dropped_summary_fixed_df.index.values:
        dropped_index_name_data = summary_fixed_df.loc[[dropped_index_name]]
        replace_name = dropped_index_name
        for all_index_name in summary_fixed_df.index.values:
            all_index_name_data = summary_fixed_df.loc[[all_index_name]]
            if(dropped_index_name == all_index_name):
                pass
            elif(evaluate_dataframes(dropped_index_name_data, all_index_name_data)):
                replace_name += f", {all_index_name}"
        dropped_summary_fixed_df_renamed = dropped_summary_fixed_df_renamed.rename(index={dropped_index_name: replace_name})
    
    return dropped_summary_fixed_df_renamed
    

In [ ]:
# ~~~_excludeBTSP, ~~~_onlyBTSP はそれぞれのベンチマークで取得したプロセス数
processes_excludeBTSP = [1, 2, 4, 8, 16, 32, 64, 128, 256]
processes_onlyBTSP = [1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 169, 196, 225]

# 引数に横軸：プロセス数orベンチマーククラス, 縦軸：関数名となっているデータフレームを取る
# 返値として
# rowData:プロセス数のリスト もしくは プロセス数のリスト (引数に由来)
# 各種関数名：実行回数のリスト
# 以上のような関係の辞書を返す

def return_dict_Data(DataFrame):
    columns = DataFrame.columns.to_numpy()
    index = DataFrame.index.to_numpy()
    # 返値となる辞書return_dictに引数のデータフレームの列名(プロセス数orベンチマーククラス)を格納
    return_dict = {"rowData":columns}
    for index_name in index:
        return_dict[index_name] = DataFrame.T[index_name].to_numpy()
    
    return return_dict


### 引数はx軸となる値のリスト, y軸となる値のリスト, 関数名の文字列, 訓練データでMAPEを算出するかどうかの真偽
### 返り値は次のようなリスト
### [<関数名の文字列>, <線形モデルのMAPE値>, <対数モデルのMAPE値>, <反比例モデルのMAPE値>, <分岐モデルのMAPE値>]
def return_Mape_row_list(x :list, y :list, function_name :str, test_ratio=0.3, train=False):

    # 変数：model_lin
    # 線形モデル
    model_lin = ModelLin(x, y, test_ratio=test_ratio)
    model_lin.calc_lr()
    model_lin.calc_mape_score()
    model_lin.calc_mape_score_InTrain()

    # 変数：model_log
    # 対数モデル
    model_log = ModelLog10(x, y, test_ratio=test_ratio)
    model_log.calc_lr()
    model_log.calc_mape_score()
    model_log.calc_mape_score_InTrain()

    # 変数：model_ip
    # 反比例モデル
    model_ip = ModelIP(x, y, test_ratio=test_ratio)
    model_ip.calc_lr()
    model_ip.calc_mape_score()
    model_ip.calc_mape_score_InTrain()

    # 変数：model_branch
    # 特異点付き条件分岐モデル
    model_branch = ModelBranch(x, y, test_ratio=test_ratio)
    model_branch.calc_lr()
    model_branch.calc_mape_score()
    model_branch.calc_mape_score_InTrain()

    if(train==True):
        lin_score = model_lin.mape_score_InTrain
        log_score = model_log.mape_score_InTrain
        ip_score = model_ip.mape_score_InTrain
        branch_score = model_branch.mape_score_InTrain
    else:
        lin_score = model_lin.mape_score
        log_score = model_log.mape_score
        ip_score = model_ip.mape_score
        branch_score = model_branch.mape_score

    # 変数：return_list
    # 返り値となるリスト
    return_list = [function_name, lin_score, log_score, ip_score, branch_score]
    return(return_list)

### 引数は「return_dict_DataFrame()」の返値, テストとして何割のデータを用いるかの割合, 訓練データでMAPEを算出するかの真偽
### 返り値は行・列がモデル名・関数名で要素がMAPE値となっているDataFrame
def return_MapeTable_per_benchmark(dict_data :dict, test_ratio, train=False):

    # 変数：_names
    # 引数の辞書のプロセス数もしくはベンチマーククラスの文字列のリスト
    _names = dict_data['rowData']
    # 変数：function_names
    # 引数の辞書の関数名の文字列のリスト
    function_names = list(dict_data.keys())
    function_names.remove('rowData')

    # リスト変数：before_DataFrame_list
    # 最終的にDataFrameとする元となるリスト
    before_DataFrame_list = []
    collumn_names = ["function name", "Linear model", "Log10 model", "Inverse model", "Branch model"]
    for function_name in function_names:
        if(does_include_nan(dict_data[function_name])):
            continue
        before_DataFrame_list.append(return_Mape_row_list(x = _names,y = dict_data[function_name],function_name = function_name, test_ratio=test_ratio, train=train))
    
    # 変数：return_df
    # 返り値となるリスト
    return_df = pd.DataFrame(before_DataFrame_list)
    return_df.columns = collumn_names
    return_df = return_df.set_index("function name")

    return(return_df)


### 構造体的に利用可能なクラス MapeData
### 各ベンチマークの各モデルごとに作成される。
### 要素として、割合, 最大値, 最小値 がある。
class MapeData:
    def __init__(self):
        self.ratio = 0
        self.max = np.nan
        self.min = np.nan
        self.appearance = 0

    def printData(self):
        print(f"{self.ratio}({self.min}, {self.max})")

    def return_Data(self):
        max_min = ""
        if(self.min is np.nan):
            max_min = "(NoData)"
        else:
            max_min = f"({self.min}, {self.max})"
        return(f"{self.ratio}%{max_min}")

### 引数に「return_MapeTable_per_benchmark()」の返り値, ベンチマーク名, (オプショナル)中間データの詳細をとる
### 返り値として次のようなリストを返す
### [<線形モデルのMAPEに関する奴>, <対数モデルのMAPEに関する奴>, <反比例モデルのMAPEに関する奴>, <ベンチマーク名>]

def return_MapeTable_row(MapeDataframe_detail, benchmark_name:str):
    
    # 引数として渡されたデータフレームの行列名をindex, columnsに格納
    columns = MapeDataframe_detail.columns.to_numpy()
    index = MapeDataframe_detail.index.to_numpy()

    # この関数で返すリストの要素の準備
    MapeLin = MapeData()
    MapeLog = MapeData()
    MapeIP = MapeData()
    MapeBr = MapeData()
    return_list = [MapeLin, MapeLog, MapeIP, MapeBr, benchmark_name ]

    # 返り値のリストの各要素の値を更新
    for function_name in index:
        MapeData_per_function = MapeDataframe_detail.loc[function_name].to_list()
        min_mape = min(MapeData_per_function)
        min_mape_index = MapeData_per_function.index(min_mape)
        rounded_min_mape = int(min_mape * 10) / 10
        return_list[min_mape_index].appearance += 1
        if(return_list[min_mape_index].max is np.nan):
            return_list[min_mape_index].max = rounded_min_mape
            return_list[min_mape_index].min = rounded_min_mape
        if(return_list[min_mape_index].min > min_mape):
            return_list[min_mape_index].min = rounded_min_mape
        elif(return_list[min_mape_index].max < min_mape):
            return_list[min_mape_index].max = rounded_min_mape
    sum_num = 0
    # 集計データから割合を算出
    for i in range(return_list.index(benchmark_name)):
        sum_num += return_list[i].appearance
    for i in range(return_list.index(benchmark_name)):
        return_list[i].ratio = int(100 * return_list[i].appearance/sum_num)
    # 割合の合計が100になるように調整
    exclude_index0_ratios = 0
    for i in range(return_list.index(benchmark_name)):
        if i == 0:
            continue
        exclude_index0_ratios += return_list[i].ratio
    return_list[0].ratio = 100 - exclude_index0_ratios

    return(return_list)

def save_MapeTable(MapeTable, suffix=""):
    tmp_table = MapeTable.copy()
    columns = MapeTable.columns.to_numpy()
    index = MapeTable.index.to_numpy()
    for i in range(len(columns)):
        for j in range(len(index)):
            tmp_table.iat[j, i] = tmp_table.iat[j, i].return_Data()
    tmp_table.to_csv(f"./tmp_GenerateResources/MapeTable_{str(suffix)}.csv")

In [ ]:
### 引数はx軸となる値のリスト, y軸となる値のリスト, 関数名の文字列
### 返り値は次のようなリスト
### [<関数名の文字列>, <線形モデル>, <対数モデル>, <反比例モデル>, <分岐モデル>]
def return_Model_row_list(x :list, y :list, function_name :str, test_ratio=0.3, train=False):

    # 変数：model_lin
    # 線形モデル
    model_lin = ModelLin(x, y, test_ratio=test_ratio)
    model_lin.calc_lr()
    model_lin.calc_mape_score()
    model_lin.calc_mape_score_InTrain()

    # 変数：model_log
    # 対数モデル
    model_log = ModelLog10(x, y, test_ratio=test_ratio)
    model_log.calc_lr()
    model_log.calc_mape_score()
    model_log.calc_mape_score_InTrain()

    # 変数：model_ip
    # 反比例モデル
    model_ip = ModelIP(x, y, test_ratio=test_ratio)
    model_ip.calc_lr()
    model_ip.calc_mape_score()
    model_ip.calc_mape_score_InTrain()

    # 変数：model_branch
    # 特異点付き条件分岐モデル
    model_branch = ModelBranch(x, y, test_ratio=test_ratio)
    model_branch.calc_lr()
    model_branch.calc_mape_score()
    model_branch.calc_mape_score_InTrain()

    # 変数：return_list
    # 返り値となるリスト
    return_list = [function_name, model_lin, model_log, model_ip, model_branch]
    return(return_list)

In [ ]:
### 引数は、収集するベンチマークのリスト、実行したい学習の割合、固定したいベンチマーククラス
### 返値は無し
### 実行すると、"./tmpGenerateResources" に "<ベンチマーク名>_FixedClassTrain_<テスト割合>.csv" という形式でファイルが生成される

def GenerateMapeTableFixClass(Benchmarks=["cg"], TestRatios=["0.3"], FixBenchmarksClass="C"):
    for test_ratio in TestRatios:
        print(f"test_ratio={test_ratio}")
        print(f"train_list, test_list = {split_by_ratio(processes_excludeBTSP, test_ratio)} on processes_excludeBTSP")
        print(f"train_list, test_list = {split_by_ratio(processes_onlyBTSP, test_ratio)} on processes_onlyBTSP")
        print(f"\n")
        fixed_class_list = [0] * len(Benchmarks)
        for i in range(len(Benchmarks)):
            if(Benchmarks[i] == "bt" or Benchmarks[i] == "sp"):
                process = processes_onlyBTSP
            else:
                process = processes_excludeBTSP
            fixed_class_list[i] = return_fixed_class(BenchMark=Benchmarks[i], Processes=processes, FixedBenchMarkClass=FixBenchmarksClass)
        fixed_class_DataFrame = [0] * len(fixed_class_list)
        for i in range(len(fixed_class_list)):
            fixed_class_DataFrame[i] = pd.concat(fixed_class_list[i], axis=1)
        for i in range(len(fixed_class_DataFrame)):
            dict_data = return_dict_Data(fixed_class_DataFrame[i])
            MapeTable_per_benchmark = return_MapeTable_per_benchmark(dict_data, test_ratio=test_ratio, train=True)
            MapeTable_per_benchmark.to_csv(f"./tmp_GenerateResources/{Benchmarks[i]}_FixedClassTrain_{test_ratio}.csv")
# 使用例       
# GenerateMapeTableFixClass(Benchmarks=["cg", "lu"], TestRatios=[0.3, 0.7], FixBenchmarksClass="C")

In [ ]:
### 引数は、収集するベンチマークのリスト、実行したい学習の割合、固定したい実行プロセス数
### 返値は無し
### 実行すると、"./tmpGenerateResources" に "<ベンチマーク名>_FixedProcessTrain_<テスト割合>.csv" という形式でファイルが生成される

def GenerateMapeTableFixProcess(Benchmarks=["cg"], BenchmarkClasses=["A", "B", "C", "D"], BenchmarkClasses_on_num=[1, 4, 16, 256], TestRatios=["0.3"], FixProcess=64):
    for test_ratio in TestRatios:
        print(f"test_ratio={test_ratio}")
        print(f"train_list, test_list = {split_by_ratio(BenchmarkClasses, test_ratio)} on BenchmarkClasses")
        print(f"\n")
        fixed_Process_list = [0] * len(benchmarks)
        for i in range(len(fixed_Process_list)):
            fixed_Process_list[i] = return_fixed_process(BenchMark=Benchmarks[i], BenchMarkClasses=BenchmarkClasses, FixedProcess=FixProcess)
        fixed_Process_DataFrame = [0] * len(fixed_Process_list)
        for i in range(len(fixed_Process_DataFrame)):
            fixed_Process_DataFrame[i] = pd.concat(fixed_Process_list[i], axis=1)
            
        for i in range(len(fixed_Process_DataFrame)):
            dict_data = return_dict_Data(fixed_Process_DataFrame[i])
            dict_data['rowData'] = BenchmarkClasses_on_num
            try:
                MapeTable_per_benchmark=return_MapeTable_per_benchmark(dict_data, test_ratio=test_ratio, train=True)
            except:
                print(f"MAPEを算出するのに問題発生@{Benchmarks[i]}")
                continue
            MapeTable_per_benchmark.to_csv(f"./tmp_GenerateResources/{benchmarks[i]}_FixedProcessTrain_{test_ratio}.csv")

In [ ]:
def ConvertBencharkClass_inNPB(Alphabet :str):
    if(Alphabet == "A"):
        return (1)
    elif(Alphabet == "B"):
        return (4)
    elif(Alphabet == "C"):
        return (16)
    elif(Alphabet == "D"):
        return (256)
    else:
        return False

def ConvertBenchmarkClasses(InputList=["A", "B", "C", "D"]):
    ReturnList = []
    for content in InputList:
        ReturnList.append(ConvertBencharkClass_inNPB(content))
    return(ReturnList)

In [ ]:
def return_OptimalProcessesList(BenchmarkName="cg"):
    if(BenchmarkName=="bt" or BenchmarkName=="sp"):
        return(processes_onlyBTSP)
    else:
        return(processes_excludeBTSP)

In [ ]:
### 引数は、ベンチマーク名、列名、固定する値、プロセスorクラスのどちらで固定するか
### 返値は次のような形式の辞書
### rowData:[プロセス数]or[ベンチマーククラス]
### <関数名>:[実行回数]
def returnDictForModelDataFrame(BenchmarkName="cg", rowData=["A", "B", "C", "D"], fix="64", fixed="Process"):
    if(fixed=="Process"):
        FixedProcessList = return_fixed_process(BenchMark=BenchmarkName, BenchMarkClasses=rowData, FixedProcess=fix)
        FixedProcessDataFrame = pd.concat(FixedProcessList, axis=1)
        DictData = return_dict_Data(FixedProcessDataFrame)
    elif(fixed=="Class"):
        FixedClassList = return_fixed_class(BenchMark=BenchmarkName, Processes=rowData, FixedBenchMarkClass=fix)
        FixedClassDataFrame = pd.concat(FixedClassList, axis=1)
        DictData = return_dict_Data(FixedClassDataFrame)
    return(DictData)

In [ ]:
### 引数は、returnDictForModelDataFrame()の返値, rowData, 関数名のリスト, テストに割り当てる割合
### 返値はリスト
### [<関数名>, <学習済みデータ1>, ... , <学習済みデータn>]
def return_ModelDataSourceList(DictData, x_list, Index, test_ratio=0.3):
    ModelDataSourceList = []
    for FunctionName in Index:
        y_list = DictData[FunctionName]
        if(does_include_nan(y_list)):
            continue
        ModelDataSourceList.append(return_Model_row_list(x=x_list, y=y_list, function_name=FunctionName, test_ratio=test_ratio, train=True))
    return(ModelDataSourceList)

In [ ]:
### 引数は、関数名、rowData, テストに割り当てる割合, 固定するプロセス数orクラス, クラスで固定するかプロセス数で固定するか
### 返値はDataFrame
### 行名は関数名で、列名はモデルの種別
### それぞれの要素は学習済みデータ
def return_Models_per_Benchmark(BenchmarkName="cg", rowData=[1, 4, 16, 256], TestRate=0.3, fix="C", fixed="Class"):
    # 変数：MapeTable
    # ベンチマークのMAPE表
    # fixedが"Class"ならクラスで固定され、fixedが"Process"ならプロセス数で固定されたMAPE表を読み込んでいる。
    MapeTable = pd.read_csv(f"./tmp_GenerateResources/{BenchmarkName}_Fixed{fixed}Train_{test_ratio}.csv")
    MapeTable = MapeTable.set_index("function name")
    
    # 変数：MapeTableColumns, MapTableIndex
    # MapeTableの列名・行名
    MapeTableColumns = MapeTable.columns.to_numpy()
    MapeTableIndex = MapeTable.index.to_numpy()
    
    # 変数：ModelDataFrame
    # MapeTableにおける各関数の学習済みモデルが格納される
    checked_rowData=rowData
    if(fixed=="Process"):
        checked_rowData=ConvertBenchmarkClasses(rowData)
    DictData = returnDictForModelDataFrame(BenchmarkName, rowData=rowData, fix=fix, fixed=fixed)
    ModelDataFrameSourceList = return_ModelDataSourceList(DictData=DictData, x_list=checked_rowData, Index=MapeTableIndex, test_ratio=TestRate)
    ModelDataFrameSourceListCollumnsName = ["FunctionName", "ModelLin", "ModelLog", "ModelIp", "ModelBranch"]
    ModelDataFrame = pd.DataFrame(ModelDataFrameSourceList)
    ModelDataFrame.columns = ModelDataFrameSourceListCollumnsName
    ModelDataFrame = ModelDataFrame.set_index("FunctionName")
    return(ModelDataFrame)

In [ ]:
### 引数は、読み込んだベンチマークごとのMAPE表, 各関数の全てのモデルにおける学習済みモデル
### 返値は辞書
### キーは<関数名>でバリューは<学習済みモデル>
def return_BestModelsDict(MapeTable, ModelDataFrame):
    BestModelsDict = {}
    ModelNames = ModelDataFrame.columns.to_list()
    ModelDataFrameIndexNameList = ModelDataFrame.index.to_numpy()
    for FunctionName in ModelDataFrameIndexNameList:
        MapeInFunction = MapeTable.loc[FunctionName].to_list()
        SmallestModelIndex = MapeInFunction.index(min(MapeInFunction))
        SmallestModelName = ModelNames[SmallestModelIndex]
        BestModelsDict[FunctionName] = ModelDataFrame.at[FunctionName, SmallestModelName]
    return BestModelsDict

In [ ]:
BenchmarkClasses =["A", "B", "C", "D"]

### 引数は
### 返値は辞書
### キーは<関数名>でバリューは<学習済みモデル>の辞書
### 学習済みモデルのデータフレームの取得に失敗した場合はFalseを返す
def generateBestModelDict(TestRatio=0.3, BenchmarkName="ft", fixed="Class", fix="C", rowData=[1, 2, 4, 8, 16, 32, 64, 128, 256]):
    
    # 変数：MapeTable
    # ベンチマークのMAPE表
    if(fixed=="Class"):
        file_name = f"{BenchmarkName}_FixedClassTrain_{TestRatio}.csv"
    else:
        file_name = f"{BenchmarkName}_FixedProcessTrain_{TestRatio}.csv"
    file_path = f"./tmp_GenerateResources/{file_name}"
    MapeTable = pd.read_csv(file_path)
    MapeTable = MapeTable.set_index("function name")
#     try:
#         ModelDataFrame = return_Models_per_Benchmark(BenchmarkName=benchmark, rowData=processes, TestRate=test_ratio, fix=fix, fixed=fixed)
#     except:
#         print(f"\n全てのモデル形式で学習済みモデルを作成しているor集めている最中に問題が発生しました@{benchmark}\n")
#         return False
    ModelDataFrame = return_Models_per_Benchmark(BenchmarkName=benchmark, rowData=rowData, TestRate=test_ratio, fix=fix, fixed=fixed)

    BestModelsDict = return_BestModelsDict(MapeTable=MapeTable, ModelDataFrame=ModelDataFrame)
    return(BestModelsDict)

In [ ]:
# 引数は、テストの割合のリスト, rowData
# 返値はrowDataのtrainのリストを文字列化したもののリスト
def return_StringTrainList(TestRatio=[0.3, 0.5], rowData=[1,2,4,8]):
    returnList = []
    for test_ratio in TestRatio:
        train_list, test_list = split_by_ratio(base_list=rowData, test_ratio=test_ratio)
        returnList.append(f"{train_list}")
    return(returnList)

In [ ]:
# 引数は、関数名, 予測値, ベンチマーク名, ベンチマーククラス, 実行プロセス数
# 返値は予測値と実測値の誤差率
def return_ErrorRate(FunctionName="CG", PredictNum=256, BenchmarkName="cg", BenchmarkClass="D", Process=256):
    target_csv = pd.read_csv(f"./csv_files/pprof_{BenchmarkName}{BenchmarkClass}{Process}.csv")
    target_csv = target_csv.set_index("Name")
    try:
        RealNum = target_csv.loc[FunctionName, "#Call"]
    except:
        print(f"該当する関数はありませんでした@{Benchmakname}@{FunctionName}")
        RealNum = False
    if(RealNum != False):
        returnNum = abs(RealNum-PredictNum)/RealNum
        return(returnNum)
    else:
        return False

In [ ]:
def return_Predicted(LearnedModel, num):
    Input = np.reshape(num, (-1, 1))
    PredictedData = LearnedModel.predict(Input)
    PredictedList = PredictedData.tolist()
    Predict = PredictedList[0][0]
    return(Predict)